In [22]:
import cv2
import numpy as np 
from skimage.feature import hog
from sklearn.svm import LinearSVC
import pickle
# import pandas as pd

pickle_in = open('OCR.pickle','rb') 
clf = pickle.load(pickle_in)

im = cv2.imread("photo_1.jpg")
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray,(5,5),0)
# to apply smoothning to the grayscale image 
ret, im_th = cv2.threshold(im_gray,90,255,cv2.THRESH_BINARY_INV)
#                          sourse,thresh,maxval,type
#now we are finding the contours of the image - containers and regions of the image
# now we are going to find the contours of the image to find the continuous areas of the image 
ctrs,heir = cv2.findContours(im_th.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#                              image,mode,method,Contours
#contours and heirarchy - we are applying the copy of the thresholded image to the contours
#now we are going to draw rectangle on the characters
rects = [cv2.boundingRect(ctr) for ctr in ctrs]
#here we have applied a loop to draw a rectangle around every contour of all the contours
for rect in rects:
    cv2.rectangle(im,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,255,0),3)
    # i am drawing the rectangle on the original image so the source image is 'im'
    # now we have to define the points on which the rectangle is to be made like (x1,y1) and (x2,y2)
    # here (x2,y2) are the extended versions of the rectangle boundary for automatically adjusting the size of the rectangle
    # according to the size of the character
    # now we are going to define the color of the rectangle - green color(0,255,0) and width of the line is '3'
    leng = int(rect[3]*1.6)
    # this length is coming from the third rectangle
    pt1 = int(rect[1]+rect[3] // 2 - leng // 2)
    pt2 = int(rect[0]+rect[2] // 2 - leng // 2)
    # now we are going to apply these on the region of interst
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
              # changes on the x-axis and y-axis
    # roi = cv2.resize(roi , (28,28) , interpolation=cv2.INTER_AREA )
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    # resizing the image looses some components of the image so now we have to recover those damages we have to dilate the image
    roi = cv2.dilate(roi,(3,3))
    roi_hog_fd = hog(roi, orientations=9 , pixels_per_cell=(14,14),cells_per_block=(1, 1), visualize=False)
    nbr = clf.predict(np.array([roi_hog_fd],'float64'))
    # nbr variabel is doing the predictions
    cv2.putText(im,str(int(nbr[0])),(rect[0],rect[1]),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,255),3)

cv2.imshow("Resulting image with ROIs ",im)
cv2.waitKey(0)
cv2.destroyAllWindows()
    
    
    

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [21]:
import cv2
import numpy as np 
from skimage.feature import hog
from sklearn.svm import LinearSVC
import pickle
# import pandas as pd

pickle_in = open('OCR.pickle','rb') 
clf = pickle.load(pickle_in)

def resize2SquareKeepingAspectRation(img, size, interpolation):
  h, w = img.shape[:2]
  c = None if len(img.shape) < 3 else img.shape[2]
  if h == w: return cv2.resize(img, (size, size), interpolation)
  if h > w: dif = h
  else:     dif = w
  x_pos = int((dif - w)/2.)
  y_pos = int((dif - h)/2.)
  if c is None:
    mask = np.zeros((dif, dif), dtype=img.dtype)
    mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
  else:
    mask = np.zeros((dif, dif, c), dtype=img.dtype)
    mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]
  return cv2.resize(mask, (size, size), interpolation)

im = cv2.imread("photo_6.jpg")
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray,(5,5),0)
# to apply smoothning to the grayscale image 
ret, im_th = cv2.threshold(im_gray,90,255,cv2.THRESH_BINARY_INV)
#                          sourse,thresh,maxval,type
#now we are finding the contours of the image - containers and regions of the image
# now we are going to find the contours of the image to find the continuous areas of the image 
ctrs,heir = cv2.findContours(im_th.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#                              image,mode,method,Contours
#contours and heirarchy - we are applying the copy of the thresholded image to the contours
#now we are going to draw rectangle on the characters
rects = [cv2.boundingRect(ctr) for ctr in ctrs]
#here we have applied a loop to draw a rectangle around every contour of all the contours
for rect in rects:
    cv2.rectangle(im,(rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,255,0),3)
    # i am drawing the rectangle on the original image so the source image is 'im'
    # now we have to define the points on which the rectangle is to be made like (x1,y1) and (x2,y2)
    # here (x2,y2) are the extended versions of the rectangle boundary for automatically adjusting the size of the rectangle
    # according to the size of the character
    # now we are going to define the color of the rectangle - green color(0,255,0) and width of the line is '3'
    leng = int(rect[3]*1.6)
    # this length is coming from the third rectangle
    pt1 = int(rect[1]+rect[3] // 2 - leng // 2)
    pt2 = int(rect[0]+rect[2] // 2 - leng // 2)
    # now we are going to apply these on the region of interst
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
              # changes on the x-axis and y-axis
    #roi = cv2.resize(roi, (28,28), interpolation = cv2.INTER_AREA)
    roi = resize2SquareKeepingAspectRation(roi , 28 ,cv2.INTER_AREA)
    # resizing the image looses some components of the image so now we have to recover those damages we have to dilate the image
    roi = cv2.dilate(roi,(3,3))
    roi_hog_fd = hog(roi, orientations=9 , pixels_per_cell=(14,14),cells_per_block=(1, 1), visualize=False)
    nbr = clf.predict(np.array([roi_hog_fd],'float64'))
    # nbr variabel is doing the predictions
    cv2.putText(im,str(int(nbr[0])),(rect[0],rect[1]),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,255),3)

cv2.imshow("Resulting image with ROIs ",im)
cv2.waitKey(0)
cv2.destroyAllWindows()
    
    
    